<a href="https://colab.research.google.com/github/EdwardFang09/IEE4912/blob/main/whisper_benchmark_(documentation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Benchmark (no early stopping)

Suara direkam di lab ramai untuk simulasi.

In [ ]:
#Library for quick-start on google colab
!pip install faster-whisper jiwer nvidia-ml-py3

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.1 MB/s eta 0:00:00
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19173 sha256=6c54c8126995f4aeb55948051e07eb92cbf8a3bdfbfd40780fa2ed80e23f9733
  Stored in directory: /root/.cache/pip/wheels/47/50/9e/29dc79037d74c3c1bb4a8661fb608e8674b7e4260d6a3f8f51
Successfully built nvidia-ml-py3


In [ ]:
import time
from faster_whisper import WhisperModel, BatchedInferencePipeline
import pandas as pd
import jiwer
import torch
from jiwer import transforms
import nvidia_smi

# Define audio file and its ground truth transcript
audio_file = "edwardmentimeter.m4a"  # Replace with your audio file
ground_truth_transcript = "sora, open mentimeter"  # Replace with the actual transcript

# Model sizes and compute types to benchmark
model_configs = [
    {"size": "tiny", "compute": "float32"},
    {"size": "tiny", "compute": "float16"},
    {"size": "tiny", "compute": "int8"},  # CPU INT8
    {"size": "tiny", "compute": "int8_float16"}, # GPU INT8
    {"size": "base", "compute": "float32"},
    {"size": "base", "compute": "float16"},
    {"size": "base", "compute": "int8_float16"}, # GPU INT8
    {"size": "small", "compute": "float32"},
    {"size": "small", "compute": "float16"},
    {"size": "small", "compute": "int8_float16"}, # GPU INT8
    {"size": "medium", "compute": "float32"},
    {"size": "medium", "compute": "float16"},
    {"size": "medium", "compute": "int8_float16"}, # GPU INT8
    {"size": "large-v2", "compute": "float32"},
    {"size": "large-v2", "compute": "float16"},
    {"size": "large-v2", "compute": "int8_float16"}, # GPU INT8
    {"size": "large-v3", "compute": "float32"},
    {"size": "large-v3", "compute": "float16"},
    {"size": "large-v3", "compute": "int8_float16"}, # GPU INT8
    {"size": "turbo", "compute": "float16"},  # Turbo only supports FP16
]

batch_sizes =  [8, 16, 32]# Experiment with batch sizes

results = []

for config in model_configs:
    for batch_size in batch_sizes:
        if config["size"] in ["base", "small", "medium", "large-v2", "large-v3"]:
            effective_batch_size = min(batch_size, 4)  # Reduce for larger models
        elif config["size"] == "turbo":
             effective_batch_size = min(batch_size, 8)
        else:
            effective_batch_size = batch_size

        start_time = time.time()

        try:
            device = "cuda" if config["compute"]!= "int8" else "cpu"
            model = WhisperModel(config["size"], device=device, compute_type=config["compute"])

            if device == "cuda":
                nvidia_smi.nvmlInit()
                handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
                # No memory info printed here

            if config["size"] == "turbo":
                batched_model = BatchedInferencePipeline(model=model)
                segments, info = batched_model.transcribe(audio_file, batch_size=effective_batch_size, language='en')
            else:
                segments, info = model.transcribe(audio_file, beam_size=5, language='en')

            segments_list = list(segments)

            transform = transforms.Compose([
                transforms.ToLowerCase(),
                transforms.RemovePunctuation(),
                transforms.RemoveMultipleSpaces(),
                transforms.Strip(),
            ])

            ground_truth_transformed = transform(ground_truth_transcript)
            predicted_transcript = " ".join([segment.text for segment in segments_list])
            predicted_transcript_transformed = transform(predicted_transcript)


            wer = jiwer.wer(ground_truth_transformed, predicted_transcript_transformed)

            end_time = time.time()
            inference_time = end_time - start_time

            if device == "cuda":
                infoo = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)

            results.append({
                "model_size": config["size"],
                "compute_type": config["compute"],
                "batch_size": effective_batch_size,
                "inference_time": inference_time,
                "language": info.language,
                "language_probability": info.language_probability,
                "num_segments": len(segments_list),
                "wer": wer,
                "predicted_transcript": predicted_transcript, #added predicted transcript
                "Used memory (GB)": infoo.used / (1024 ** 3),
            })

            print(f"Model: {config['size']}, Compute: {config['compute']}, Batch: {effective_batch_size}, Time: {inference_time:.2f}s, WER: {wer:.2f}, Text:, {predicted_transcript}, Used memory (GB): {infoo.used / (1024 ** 3)}") # added predicted transcript to the output

        except Exception as e:
            print(f"Error with Model: {config['size']}, Compute: {config['compute']}, Batch: {effective_batch_size}: {e}")
            results.append({
                "model_size": config["size"],
                "compute_type": config["compute"],
                "batch_size": effective_batch_size,
                "inference_time": "Error",
                "error": str(e),
                "wer": "Error",
                "predicted_transcript": "Error" # added predicted transcript in case of error
            })

        finally:
            del model
            if 'batched_model' in locals():
                del batched_model
            torch.cuda.empty_cache()

# Print or save results (e.g., to a CSV file)
df = pd.DataFrame(results)
df.to_csv("no_stop_whisper_benchmark.csv", index=False)
print(df)

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

Model: tiny, Compute: float32, Batch: 8, Time: 3.88s, WER: 2.67, Text:,  Yeah, I think so.  Yeah, I think so., Used memory (GB): 0.5604248046875
Model: tiny, Compute: float32, Batch: 16, Time: 1.24s, WER: 3.00, Text:,  So, now we are going to connect the metter., Used memory (GB): 0.5604248046875
Model: tiny, Compute: float32, Batch: 32, Time: 0.98s, WER: 2.33, Text:,  I'm so happy.  So, I'm a 20m., Used memory (GB): 0.5604248046875
Model: tiny, Compute: float16, Batch: 8, Time: 1.12s, WER: 2.67, Text:,  I am so happy.  I am so happy., Used memory (GB): 0.5936279296875
Model: tiny, Compute: float16, Batch: 16, Time: 0.77s, WER: 2.33, Text:,  So, now we're going to connect it., Used memory (GB): 0.5311279296875
Model: tiny, Compute: float16, Batch: 32, Time: 0.78s, WER: 2.00, Text:,  Yes, sorry.  You are a 90-meter., Used memory (GB): 0.6248779296875
Model: tiny, Compute: int8, Batch: 8, Time: 3.49s, WER: 3.00, Text:,  So, now we are going to connect the metter., Used memory (GB): 0.624

model.bin:   0%|          | 0.00/145M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.31k [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.71M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

Model: turbo, Compute: float16, Batch: 8, Time: 10.50s, WER: 1.67, Text:,  I'm sorry about 20 meters, Used memory (GB): 2.0643310546875
Model: turbo, Compute: float16, Batch: 8, Time: 2.75s, WER: 1.67, Text:,  I'm sorry about 20 meters, Used memory (GB): 2.0643310546875
Model: turbo, Compute: float16, Batch: 8, Time: 2.15s, WER: 1.67, Text:,  I'm sorry about 20 meters, Used memory (GB): 2.0643310546875
   model_size  compute_type  batch_size  inference_time language  \
0        tiny       float32           8        3.881012       en   
1        tiny       float32          16        1.241115       en   
2        tiny       float32          32        0.983896       en   
3        tiny       float16           8        1.123452       en   
4        tiny       float16          16        0.767862       en   
5        tiny       float16          32        0.776328       en   
6        tiny          int8           8        3.490858       en   
7        tiny          int8          16        3.5

Kesimpulan: tidak perlu model besar. Model small bahkan sudah cukup untuk akurasi 100% dengan pemakaian memori ~1GB juga dengan waktu proses <2 detik

<h1 >with early stopping (all models)

In [ ]:
import time
from faster_whisper import WhisperModel, BatchedInferencePipeline
import pandas as pd
import jiwer
import nvidia_smi
from jiwer import transforms
import os
import torch

# Define audio files and their ground truth transcripts (dictionary)
audio_ground_truth = {
    "edwardchrome.m4a": "hey sora, open chrome.",
    "edwardkahoot.m4a": "hey sora, open kahoot.",
    #... more audio files and transcripts
}

# Model sizes and compute types to benchmark
model_configs = [
    {"size": "tiny", "compute": "float32"},
    {"size": "tiny", "compute": "float16"},
    {"size": "tiny", "compute": "int8"},  # CPU INT8
    {"size": "base", "compute": "float32"},
    {"size": "base", "compute": "float16"},
    {"size": "small", "compute": "float32"},
    {"size": "small", "compute": "float16"},
    {"size": "medium", "compute": "float32"},
    {"size": "medium", "compute": "float16"},
    {"size": "turbo", "compute": "float16"},  # Turbo only supports FP16
    {"size": "large-v2", "compute": "float32"},
    {"size": "large-v2", "compute": "float16"},
    {"size": "large-v3", "compute": "float32"},
    {"size": "large-v3", "compute": "float16"},
]

results = []

print(f"CUDA available: {torch.cuda.is_available()}")

i = 0
for audio_file, ground_truth_transcript in audio_ground_truth.items():
    for config in model_configs:
        start_time = time.time()

        try:
            device = "cuda" if config["compute"]!= "int8" else "cpu"

            if device == "cuda":
                nvidia_smi.nvmlInit()
                handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
                # No memory info printed here

            model = WhisperModel(config["size"], device=device, compute_type=config["compute"])

            if config["size"] == "turbo":
                batched_model = BatchedInferencePipeline(model=model)
                segments, info = batched_model.transcribe(audio_file, language='en')
            else:
                segments, info = model.transcribe(audio_file, beam_size=5, language='en')

            segments_list = list(segments)

            transform = transforms.Compose([
                transforms.ToLowerCase(),
                transforms.RemovePunctuation(),
                transforms.RemoveMultipleSpaces(),
                transforms.Strip(),
            ])

            ground_truth_transformed = transform(ground_truth_transcript)
            predicted_transcript = " ".join([segment.text for segment in segments_list])
            predicted_transcript_transformed = transform(predicted_transcript)

            wer = jiwer.wer(ground_truth_transformed, predicted_transcript_transformed)

            end_time = time.time()
            inference_time = end_time - start_time

            if device == "cuda":
                infoo = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)

            results.append({
                "audio_file": audio_file,
                "model_size": config["size"],
                "compute_type": config["compute"],
                "inference_time": inference_time,
                "language": info.language,
                "language_probability": info.language_probability,
                "num_segments": len(segments_list),
                "wer": wer,
                "predicted_transcript": predicted_transcript,
                "Used memory (GB)": infoo.used / (1024 ** 3)
            })

            print(
                f"Audio: {audio_file}, Model: {config['size']}, Compute: {config['compute']}, Time: {inference_time:.2f}s, WER: {wer:.2f}, Text:, {predicted_transcript}, Used memory (GB): {infoo.used / (1024 ** 3)}"
            )

            if wer == 0:
                if i == 0:
                  i += 1 #add one more iteration for safety measure
                else:
                  print(f"Skipping remaining models for this audio")
                  break

        except Exception as e:
            print(
                f"Error with Audio: {audio_file}, Model: {config['size']}, Compute: {config['compute']}: {e}"
            )
            results.append({
                "audio_file": audio_file,
                "model_size": config["size"],
                "compute_type": config["compute"],
                "inference_time": "Error",
                "error": str(e),
                "wer": "Error",
                "predicted_transcript": "Error",
            })

        finally:
            del model
            if config["size"] == "turbo":
                del batched_model
            torch.cuda.empty_cache()

# Print or save results (e.g., to a CSV file)
df = pd.DataFrame(results)
df.to_csv("whisper_benchmark_results.csv", index=False)
print(df)

CUDA available: True
Audio: edwardjamboard.wav, Model: tiny, Compute: float32, Time: 1.24s, WER: 0.50, Text:,  Hesora Open Jamboard., Used memory (GB): 0.5643310546875
Audio: edwardjamboard.wav, Model: tiny, Compute: float16, Time: 0.73s, WER: 0.75, Text:,  Hesora Open Jambod., Used memory (GB): 0.5330810546875
Audio: edwardjamboard.wav, Model: tiny, Compute: int8, Time: 1.83s, WER: 0.75, Text:,  Hesora Open Jambord., Used memory (GB): 0.5330810546875
Audio: edwardjamboard.wav, Model: base, Compute: float32, Time: 1.38s, WER: 0.25, Text:,  Hey Sora, Open Jambore!, Used memory (GB): 0.6893310546875
Audio: edwardjamboard.wav, Model: base, Compute: float16, Time: 0.59s, WER: 0.25, Text:,  Hey Sora, Open Jambore!, Used memory (GB): 0.5955810546875
Audio: edwardjamboard.wav, Model: small, Compute: float32, Time: 4.11s, WER: 0.00, Text:,  Hey Sora! Open Jamboard!, Used memory (GB): 1.3143310546875
Audio: edwardjamboard.wav, Model: small, Compute: float16, Time: 0.98s, WER: 0.00, Text:,  Hey 

<h1> turbo model (special model - optimized large)

In [ ]:
import time
from faster_whisper import WhisperModel, BatchedInferencePipeline
import pandas as pd
import jiwer
import nvidia_smi
from jiwer import transforms
import os
import torch

# Define audio files and their ground truth transcripts (dictionary)
audio_ground_truth = {
    "edwardchrome.m4a": "hey sora, open chrome.",
    "edwardkahoot.m4a": "hey sora, open kahoot.",
    #... more audio files and transcripts
}

# Model sizes and compute types to benchmark
model_configs = [
    {"size": "turbo", "compute": "float16"},  # Turbo only supports FP16
]

results = []

print(f"CUDA available: {torch.cuda.is_available()}")

for audio_file, ground_truth_transcript in audio_ground_truth.items():
    for config in model_configs:
        start_time = time.time()

        try:
            device = "cuda" if config["compute"]!= "int8" else "cpu"

            if device == "cuda":
                nvidia_smi.nvmlInit()
                handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
                # No memory info printed here

            model = WhisperModel(config["size"], device=device, compute_type=config["compute"])

            if config["size"] == "turbo":
                batched_model = BatchedInferencePipeline(model=model)
                segments, info = batched_model.transcribe(audio_file, language='en')
            else:
                segments, info = model.transcribe(audio_file, beam_size=5, language='en')

            segments_list = list(segments)

            transform = transforms.Compose([
                transforms.ToLowerCase(),
                transforms.RemovePunctuation(),
                transforms.RemoveMultipleSpaces(),
                transforms.Strip(),
            ])

            ground_truth_transformed = transform(ground_truth_transcript)
            predicted_transcript = " ".join([segment.text for segment in segments_list])
            predicted_transcript_transformed = transform(predicted_transcript)

            wer = jiwer.wer(ground_truth_transformed, predicted_transcript_transformed)

            end_time = time.time()
            inference_time = end_time - start_time

            if device == "cuda":
                infoo = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)

            results.append({
                "audio_file": audio_file,
                "model_size": config["size"],
                "compute_type": config["compute"],
                "inference_time": inference_time,
                "language": info.language,
                "language_probability": info.language_probability,
                "num_segments": len(segments_list),
                "wer": wer,
                "predicted_transcript": predicted_transcript,
                "Used memory (GB)": infoo.used / (1024 ** 3)
            })

            print(
                f"Audio: {audio_file}, Model: {config['size']}, Compute: {config['compute']}, Time: {inference_time:.2f}s, WER: {wer:.2f}, Text:, {predicted_transcript}, Used memory (GB): {infoo.used / (1024 ** 3)}"
            )

            if wer == 0:
                print(f"Skipping remaining models for this audio")
                break

        except Exception as e:
            print(
                f"Error with Audio: {audio_file}, Model: {config['size']}, Compute: {config['compute']}: {e}"
            )
            results.append({
                "audio_file": audio_file,
                "model_size": config["size"],
                "compute_type": config["compute"],
                "inference_time": "Error",
                "error": str(e),
                "wer": "Error",
                "predicted_transcript": "Error",
            })

        finally:
            del model
            if config["size"] == "turbo":
                del batched_model
            torch.cuda.empty_cache()

# Print or save results (e.g., to a CSV file)
df = pd.DataFrame(results)
df.to_csv("whisper_benchmark_results_turbo.csv", index=False)
print(df)

CUDA available: True
Audio: edwardjamboard.wav, Model: turbo, Compute: float16, Time: 2.27s, WER: 0.00, Text:,  Hey Sora, open Jamboard., Used memory (GB): 2.0955810546875
Skipping remaining models for this audio
Audio: edwardchrome.m4a, Model: turbo, Compute: float16, Time: 2.29s, WER: 0.00, Text:,  Hey Sora, open Chrome., Used memory (GB): 2.0643310546875
Skipping remaining models for this audio
Audio: edwardkahoot.m4a, Model: turbo, Compute: float16, Time: 2.14s, WER: 0.25, Text:,  Hey Sora, open kahut., Used memory (GB): 2.0643310546875
           audio_file model_size compute_type  inference_time language  \
0  edwardjamboard.wav      turbo      float16        2.268531       en   
1    edwardchrome.m4a      turbo      float16        2.286799       en   
2    edwardkahoot.m4a      turbo      float16        2.141664       en   

   language_probability  num_segments   wer       predicted_transcript  \
0                     1             1  0.00   Hey Sora, open Jamboard.   
1       

Kesimpulan: turbo tidak digunakan karena memakan memori >2GB, yaitu melebihi kapasitas jetson nano 2GB

<h1 >Trained benchmark

In [ ]:
!pip install transformers torch jiwer datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:

import time
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset
import jiwer
from jiwer import transforms
import librosa
import numpy as np

# Load the processor and model
processor = WhisperProcessor.from_pretrained("EdwardFang09/whisper-base-TA-2025-v2")
model = WhisperForConditionalGeneration.from_pretrained("EdwardFang09/whisper-base-TA-2025").to("cuda")

# Load your audio data and ground truth transcripts (replace with your data)
audio_ground_truth = {
    "edwardchrome.m4a": "hey sora, open chrome.",
    "edwardkahoot.m4a": "hey sora, open kahoot.",
    # ... more audio files and transcripts
}

results = []

for audio_file, ground_truth_transcript in audio_ground_truth.items():
    start_time = time.time()

    try:
        # Load audio data using librosa
        audio_data, sr = librosa.load(audio_file, sr=16000)  # Load audio at 16kHz

        # Use the audio data (NumPy array) as input to the processor
        input_features = processor(audio_data, sampling_rate=sr, return_tensors="pt").input_features.to("cuda")

        # Generate predictions
        predicted_ids = model.generate(input_features)
        predicted_transcript = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

        # Calculate WER
        transform = transforms.Compose([
            transforms.ToLowerCase(),
            transforms.RemovePunctuation(),
            transforms.RemoveMultipleSpaces(),
            transforms.Strip(),
        ])

        ground_truth_transformed = transform(ground_truth_transcript)
        predicted_transcript_transformed = transform(predicted_transcript)
        wer = jiwer.wer(ground_truth_transformed, predicted_transcript_transformed)

        end_time = time.time()
        inference_time = end_time - start_time

        if device == "cuda":
                infoo = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)

        results.append({
            "audio_file": audio_file,
            "inference_time": inference_time,
            "wer": wer,
            "predicted_transcript": predicted_transcript,
        })

        print(f"Audio: {audio_file}, Time: {inference_time:.2f}s, WER: {wer:.2f}, Text: {predicted_transcript}")

    except Exception as e:
        print(f"Error with Audio: {audio_file}: {e}")
        results.append({
            "audio_file": audio_file,
            "inference_time": "Error",
            "error": str(e),
            "wer": "Error",
            "predicted_transcript": "Error",
        })

# Save or print the results
import pandas as pd
df = pd.DataFrame(results)
df.to_csv("huggingface_whisper_benchmark_trained.csv", index=False)
print(df)

Audio: edwardjamboard.wav, Time: 0.11s, WER: 0.75, Text: sora


<ipython-input-12-bd3c5db6ee6f>:29: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sr = librosa.load(audio_file, sr=16000)  # Load audio at 16kHz
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Audio: edwardchrome.m4a, Time: 0.21s, WER: 1.00, Text: kahoot
Audio: edwardkahoot.m4a, Time: 0.20s, WER: 0.75, Text: sora


<ipython-input-12-bd3c5db6ee6f>:29: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sr = librosa.load(audio_file, sr=16000)  # Load audio at 16kHz
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


           audio_file  inference_time   wer predicted_transcript
0  edwardjamboard.wav        0.111058  0.75                 sora
1    edwardchrome.m4a        0.212347  1.00               kahoot
2    edwardkahoot.m4a        0.201498  0.75                 sora


Dengan dataset yang sama, perhatikan perbedaan dengan model biasa.

Solusi:
1. Model di finetune supaya akurasi meningkat. <2 detik sudah relatif cepat.
  - contoh: kahut, kehut, kuhut, dll.
2. Cari banyak data untuk train dan testing. Pakai mikrofon ampas.


need a code that fetch a whole data inside a folder

optimized

In [ ]:
torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15095MB, multi_processor_count=40, uuid=5c0728cf-d769-745b-017f-61af5f802743, L2_cache_size=4MB)

In [ ]:
torch.cuda.get_device_properties(0).total_memory

15828320256

penemuan: turbo mendeteksi aksen bahasa indonesia walau sudah dicoding english dan berbicara bahasa inggris.